In [1]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge

In [2]:
def get_derivatives(u, x_points, deg = 3, order = 1):
    
    """
    Interpolates input function u with a polynomial fit of degree "deg",
    using the available evaluation points x_points. 
    Takes the derivatives up to order "order" of the polynomial.

    Input: 
    u        = values of the function on the grid
    x_points = (available) points of the grid
    deg      = polynomial interpolation degree
    order    = order of derivatives

    Output: 
    derivatives = list containing derivatives up to order "order",
                  evaluated in the middle point of "x_points"
    """
    
    # Interpolate 
    coeffs = np.polyfit(x_points, u, deg)
    poly   = np.poly1d(coeffs)
    
    # Derivatives 
    derivatives      = []
    num_points       = len(x_points)
    derivation_point = x_points[(num_points-1)//2]
    
    for i in range(1, order + 1):
        derivatives.append(poly.deriv(m=i)(derivation_point))
        
    return derivatives

In [3]:
def construct_theta(values, derivatives, derivatives_tags, deg):
    """
    Constructs a matrix Theta with columns representing the values of all admissible 
    PDE terms, i.e. with polynomial degree up to "deg", and in combination with the 
    provided derivatives. 
    It also returns a list of tags (names) that describe the columns of Theta.

    Input:
    values           = n x k array containing the values of the standalone variables
                    with each column corresponding to one variable
    derivatives      = n x l array of derivatives of the variables
    derivatives_tags = List of (string) tags for the columns of "derivatives"
    deg              = Maximum degree of polynomial functions of the variables to include in Theta

    Output:
    Theta = Combined matrix of derivatives and polynomial terms.
    tags  = List of (string) tags for the columns in Theta.  
    """
    n, K  = values.shape      # K = number of variables
    m, L  = derivatives.shape # L = number of provided derivatives

    if n != m:
        raise ValueError('Dimensions of values and derivatives do not match!')

    # Generate all polynomial combinations of degrees up to deg
    polynomials = list(itertools.product(range(deg + 1), repeat=K))
    polynomials = [p for p in polynomials if 0 < sum(p) <= deg]

    # Initialize Theta with a column of ones and corresponding tag
    Theta = np.ones((n, 1), dtype=np.float64)
    tags  = ['constant']

    # Add derivatives to Theta
    Theta = np.hstack([Theta, derivatives[:, 1:]])
    tags.extend(derivatives_tags[1:])

    # Add polynomial terms multiplied by derivatives
    for poly in polynomials:
        poly_values = np.prod(values**np.array(poly), axis=1, keepdims=True)
        for i in range(L):
            new_column = poly_values * derivatives[:, i:i + 1]
            Theta = np.hstack([Theta, new_column])
            tags.append(f"poly{poly}" +derivatives_tags[i])

    return Theta, tags


In [4]:
def print_terms(xi, tags):
    '''
    Prints all terms of the PDE's operator D
    with respective coefficients
    (PDE): u_t = D(u,u_{x},u_{xx},u^2, etc.)
    '''
    print("D = ")
    for i in range(len(xi)):
        if xi[i] != 0:
            print(xi[i], tags[i])


In [5]:
def sequential_ridge(Theta, ut, lambda_val, tol, max_iter):
    """
    Sequential Threshold Ridge Regression (sequential_ridge)
    Repeats Ridge Regression while eliminating unsignificant xi elements
    in order to achieve a sparse result. 
    Input:
    Theta      =  Theta matrix (n x d)
    ut         =  Target vector for the regression (ut = Theta * xi)
    lambda_val =  Regularization parameter for Ridge Regression
    max_iter   =  Maximum number of iterations 
    tol        =  Threshold tolerance
    
    Output:
    xi         =  Sparse solution for coefficients of PDE terms (d x 1)
    """
    n, d = Theta.shape

    # Normalize columns of Theta matrix
    normalizer = 1.0 / np.sqrt(np.sum(Theta**2, axis=0))
    Theta_n = Theta * normalizer
    ut = ut.flatten() 

    # First Ridge Regression
    ridge = Ridge(alpha=lambda_val, fit_intercept=False)
    ridge.fit(Theta_n, ut)
    xi = ridge.coef_

    # Iterate sequential Ridge Regressions
    for _ in range(max_iter):
        # Eliminate elements below tolerance threshold
        big_ids = np.where(abs(xi) >= tol)[0]
        if len(big_ids) == 0: # in case all elements were eliminated 
            break

        # Fit for significant elements only
        Theta_sparse = Theta_n[:, big_ids]
        ridge.fit(Theta_sparse, ut)
        xi_sparse = ridge.coef_

        # Reset xi and insert significant elements
        xi = np.zeros(d)
        xi[big_ids] = xi_sparse

    # Last Ridge Regression for sparse xi
    if len(big_ids) > 0:
        Theta_sparse = Theta_n[:, big_ids]
        ridge.fit(Theta_sparse, ut)
        xi_sparse = ridge.coef_
        xi = np.zeros(d)
        xi[big_ids] = xi_sparse

    # Ensure output shape is (d, 1)
    xi = xi.reshape(-1, 1)

    # Rescale coefficients (because of normalization)
    xi = xi * normalizer.reshape(-1, 1) 
    return xi 


In [6]:
#@title Dataset 1

# Extract PDE data from Dataset 1
dataset = np.load("1.npz")
U = dataset['u']
X = dataset['x'][:,0]
T = dataset['t'][0]
print("Datasets shapes: ", U.shape, X.shape, T.shape)
dt = T[1]-T[0]
dx = X[2]-X[1]

n = len(X)

Datasets shapes:  (256, 101) (256,) (101,)


In [7]:
#@title Dataset 1 Sampling 
np.random.seed(42) # seeding

num_x = 100  # number of sampled spatial points
num_t = 50   # number of sampled temporal points for each spatial point
num_samples = num_x * num_t
padding = 5 # must exclude boundary points for interpolation used in get_derivative()

coords = {}  # collect sample coordinates in a dictionary
j = 0

for i in range(num_x):
    x = np.random.choice(np.arange(padding,n-padding),1)[0]
    for t in range(padding, num_t - padding):
        coords[j] = [x,t]
        j += 1

N = 2*padding-1  # polynomial fitting points
deg = 4          # degree of polynomial to use

# Compute u and its derivatives up to the second order
u   = np.zeros((num_samples,1))
ut  = np.zeros((num_samples,1))
ux  = np.zeros((num_samples,1))
uxx = np.zeros((num_samples,1))

for c in coords.keys():
    # point coordinates
    [x,t] = coords[c]
    
    # value of function
    u[c] = U[x,t]
    
    # time derivatives
    ut[c] = get_derivatives(U[x,t-(N-1)//2:t+(N+1)//2], np.arange(N)*dt, deg, 1)[0]
    
    # spatial derivatives
    u_derivatives = get_derivatives(U[x-(N-1)//2:x+(N+1)//2,t], np.arange(N)*dx, deg, 2)
    
    ux[c]   = u_derivatives[0]
    uxx[c]  = u_derivatives[1]

# Construct Theta by stacking a "ones" column and the computed derivatives 
# Also provide tags of all derivatives
derivatives      = np.hstack([np.ones((num_samples,1)), ux, uxx])
derivatives_tags = ['','u_x', 'u_xx']
Theta, tags      = construct_theta(u, derivatives, derivatives_tags, 2)

print(tags)

['constant', 'u_x', 'u_xx', 'poly(1,)', 'poly(1,)u_x', 'poly(1,)u_xx', 'poly(2,)', 'poly(2,)u_x', 'poly(2,)u_xx']


In [8]:
#@title Find PDE for Dataset 1
xi = sequential_ridge(Theta,ut,10**-3,1.0,10)

print_terms(xi, tags)

D = 
[0.09978634] u_xx
[-0.99922242] poly(1,)u_x


In [9]:
#@title Dataset 2

# Extract PDE data from dataset 2
dataset = np.load("2.npz")
U = dataset['u']
X = dataset['x'][:,0]
T = dataset['t'][0]
print("Datasets shapes: ", U.shape, X.shape, T.shape)
dt = T[1]-T[0]
dx = X[2]-X[1]

n = len(X)

Datasets shapes:  (512, 201) (512,) (201,)


In [10]:
#@title Dataset 2 Sampling

num_x = 200 
num_t = 100
num_samples = num_x * num_t
padding    = 5

coords = {}
j = 0

for p in range(num_x):
    x = np.random.choice(np.arange(padding,n-padding),1)[0]
    for t in range(padding, num_t - padding):
        coords[j] = [x,t]
        j += 1

# Compute u and its derivatives up to the third order
u    = np.zeros((num_samples,1))
ut   = np.zeros((num_samples,1))
ux   = np.zeros((num_samples,1))
uxx  = np.zeros((num_samples,1))
uxxx = np.zeros((num_samples,1))

N   = 2*padding-1  
deg = 4             

for p in coords.keys():
    [x,t] = coords[p]
    u[p]  = U[x,t]
    ut[p] = get_derivatives(U[x,t-(N-1)//2:t+(N+1)//2], np.arange(N)*dt, deg, 1)[0]

    ux_derivatives = get_derivatives(U[x-(N-1)//2:x+(N+1)//2,t], np.arange(N)*dx, deg, 3)
    ux[p]   = ux_derivatives[0]
    uxx[p]  = ux_derivatives[1]
    uxxx[p] = ux_derivatives[2]

# Construct Theta 
derivatives = np.hstack([np.ones((num_samples,1)), ux, uxx, uxxx])
derivatives_tags = ['','u_x', 'u_xx','u_xxx']
Theta, tags = construct_theta(u, derivatives, derivatives_tags, 2)
print(tags)


['constant', 'u_x', 'u_xx', 'u_xxx', 'poly(1,)', 'poly(1,)u_x', 'poly(1,)u_xx', 'poly(1,)u_xxx', 'poly(2,)', 'poly(2,)u_x', 'poly(2,)u_xx', 'poly(2,)u_xxx']


In [11]:
#@title Find PDE for Dataset 2

xi = sequential_ridge(Theta,ut,10**-5,1.0,10)
print_terms(xi, tags)

D = 
[-1.05950493] u_xxx
[-6.09797082] poly(1,)u_x


In [12]:
#@title Dataset 3

dataset = np.load('3.npz')
T = dataset['t'][0,0,:]
X = dataset['x'][:,0,0]
Y = dataset['y'][0,:,0]
U = dataset['u']
V = dataset['v']
print("Dataset shapes: ", U.shape, X.shape, Y.shape, T.shape)

n = len(X)
steps = len(T)
dx = X[2]-X[1]
dy = Y[2]-Y[1]
dt = T[2]-T[1]


Dataset shapes:  (256, 256, 201) (256,) (256,) (201,)


In [13]:
#@title Dataset 3 Sampling

num_xy = 500
num_t  = 30
num_samples = num_xy * num_t
padding = 5

coords = {}
j = 0

for t in range(padding, num_t-padding):
    for p in range(num_xy):
        x = np.random.choice(np.arange(padding,n-padding),1)[0]
        y = np.random.choice(np.arange(padding,n-padding),1)[0]
        coords[j] = [x,y,t]
        j += 1

# Compute u,v and derivatives w.r.t x & y
u   = np.zeros((num_samples,1))
v   = np.zeros((num_samples,1))
ut  = np.zeros((num_samples,1))
vt  = np.zeros((num_samples,1))
ux  = np.zeros((num_samples,1))
uy  = np.zeros((num_samples,1))
uxx = np.zeros((num_samples,1))
uxy = np.zeros((num_samples,1))
uyy = np.zeros((num_samples,1))
vx  = np.zeros((num_samples,1))
vy  = np.zeros((num_samples,1))
vxx = np.zeros((num_samples,1))
vxy = np.zeros((num_samples,1))
vyy = np.zeros((num_samples,1))

N = 2*padding-1 
deg = 4

lin_t = np.arange(N)*dt
lin_x = np.arange(N)*dx
lin_y = np.arange(N)*dy

for p in coords.keys():
    
    [x,y,t] = coords[p]
    u[p] = U[x,y,t]
    v[p] = V[x,y,t]
    
    ux_d = get_derivatives(U[x-(N-1)//2:x+(N+1)//2,y,t], lin_x, deg, 2)
    uy_d = get_derivatives(U[x,y-(N-1)//2:y+(N+1)//2,t], lin_y, deg, 2)
    vx_d = get_derivatives(V[x-(N-1)//2:x+(N+1)//2,y,t], lin_x, deg, 2)
    vy_d = get_derivatives(V[x,y-(N-1)//2:y+(N+1)//2,t], lin_y, deg, 2)
    
    ux[p]  = ux_d[0]
    uy[p]  = uy_d[0]
    uxx[p] = ux_d[1]
    uyy[p] = uy_d[1]

    vx[p]  = vx_d[0]
    vy[p]  = vy_d[0]
    vxx[p] = vx_d[1]
    vyy[p] = vy_d[1]

    ut[p] = get_derivatives(U[x,y,t-(N-1)//2:t+(N+1)//2], lin_t, deg, 1)[0]
    vt[p] = get_derivatives(V[x,y,t-(N-1)//2:t+(N+1)//2], lin_t, deg, 1)[0]

    # construct mixed derivatives
    ux_d_y_A = get_derivatives(U[x-(N-1)//2:x+(N+1)//2,y+1,t], lin_x, deg, 2)
    ux_d_y_B = get_derivatives(U[x-(N-1)//2:x+(N+1)//2,y-1,t], lin_x, deg, 2)
    vx_d_y_A = get_derivatives(V[x-(N-1)//2:x+(N+1)//2,y+1,t], lin_x, deg, 2)
    vx_d_y_B = get_derivatives(V[x-(N-1)//2:x+(N+1)//2,y-1,t], lin_x, deg, 2)
    
    uxy[p] = (ux_d_y_A[0]-ux_d_y_B[0])/(2*dy)
    vxy[p] = (vx_d_y_A[0]-vx_d_y_B[0])/(2*dy)


In [14]:
#@title Construct Theta for Dataset 3
# Construct Theta
values = np.hstack([u,v])
derivatives = np.hstack([np.ones((num_samples,1)), ux, uy, uxx, uxy, uyy, vx, vy, vxx, vxy, vyy])
derivatives_tags = ['','u_x', 'u_y','u_xx','u_xy','u_yy','v_x', 'v_y','v_xx','v_xy','v_yy']
Theta, tags = construct_theta(values, derivatives, derivatives_tags, 3)
print(tags)


['constant', 'u_x', 'u_y', 'u_xx', 'u_xy', 'u_yy', 'v_x', 'v_y', 'v_xx', 'v_xy', 'v_yy', 'poly(0, 1)', 'poly(0, 1)u_x', 'poly(0, 1)u_y', 'poly(0, 1)u_xx', 'poly(0, 1)u_xy', 'poly(0, 1)u_yy', 'poly(0, 1)v_x', 'poly(0, 1)v_y', 'poly(0, 1)v_xx', 'poly(0, 1)v_xy', 'poly(0, 1)v_yy', 'poly(0, 2)', 'poly(0, 2)u_x', 'poly(0, 2)u_y', 'poly(0, 2)u_xx', 'poly(0, 2)u_xy', 'poly(0, 2)u_yy', 'poly(0, 2)v_x', 'poly(0, 2)v_y', 'poly(0, 2)v_xx', 'poly(0, 2)v_xy', 'poly(0, 2)v_yy', 'poly(0, 3)', 'poly(0, 3)u_x', 'poly(0, 3)u_y', 'poly(0, 3)u_xx', 'poly(0, 3)u_xy', 'poly(0, 3)u_yy', 'poly(0, 3)v_x', 'poly(0, 3)v_y', 'poly(0, 3)v_xx', 'poly(0, 3)v_xy', 'poly(0, 3)v_yy', 'poly(1, 0)', 'poly(1, 0)u_x', 'poly(1, 0)u_y', 'poly(1, 0)u_xx', 'poly(1, 0)u_xy', 'poly(1, 0)u_yy', 'poly(1, 0)v_x', 'poly(1, 0)v_y', 'poly(1, 0)v_xx', 'poly(1, 0)v_xy', 'poly(1, 0)v_yy', 'poly(1, 1)', 'poly(1, 1)u_x', 'poly(1, 1)u_y', 'poly(1, 1)u_xx', 'poly(1, 1)u_xy', 'poly(1, 1)u_yy', 'poly(1, 1)v_x', 'poly(1, 1)v_y', 'poly(1, 1)v_xx

In [18]:
#@title Find PDE for Dataset 3 (u_t)
xi = sequential_ridge(Theta,ut,10**-5,1.0,10)
print_terms(xi, tags)

D = 
[0.09859156] u_xx
[0.09871377] u_yy
[1.0001812] poly(0, 3)
[0.98838576] poly(1, 0)
[-0.98888502] poly(1, 2)
[0.99990977] poly(2, 1)
[-0.98889484] poly(3, 0)


In [17]:
#@title Find PDE for Dataset 3 (v_t)
xi = sequential_ridge(Theta,vt,10**-5,1.0,10)
print_terms(xi, tags)

D = 
[0.09857323] v_xx
[0.09873052] v_yy
[0.98672801] poly(0, 1)
[-0.98710456] poly(0, 3)
[-1.00001944] poly(1, 2)
[-0.98705056] poly(2, 1)
[-1.0001314] poly(3, 0)
